In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the labeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = dsp.labeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['explanatory','groupby'], 'year':'trend', 'driver_gender':['explanatory','groupby'], 'driver_race':['explanatory','groupby'],
       'decriminalization':['explanatory','groupby'], 'medical':['explanatory','groupby'],
         'recreational':['explanatory','groupby'], 'no_reforms':['explanatory','groupby'],
       'search_conducted_false':'trend', 'search_conducted_true':'trend',
       'search_conducted_rate':'trend', 'contraband_found_false':'trend',
       'contraband_found_true':'trend', 'contraband_found_rate':'trend', 'hit_false':'trend',
       'hit_true':'trend', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.meta_df

/home/smb/anaconda3/lib/python3.6/site-packages/detect_simpsons_paradox/labeled_dataframe.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.meta_df['role'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,trend,True,NaN


In [7]:
labeled_df_setup.meta_df.index.name

'variable'

Now, we've set this up, we can also save these configurations to load them in directly in the future

In [8]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [9]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [10]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,[trend],True,NaN


In [11]:
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [12]:
corrobj = dsp.all_pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

Index(['year', 'search_conducted_false', 'search_conducted_true',
       'search_conducted_rate', 'contraband_found_false',
       'contraband_found_true', 'contraband_found_rate', 'hit_false',
       'hit_true', 'hit_rate', 'num_stops'],
      dtype='object', name='variable')

In [13]:
rankobj = dsp.mean_rank_trend()
linreg_obj = dsp.linear_trend()

# Computing Trends on a labeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [14]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,group_feat,subgroup,trend_type,agg_trend
0,year,search_conducted_false,0.299408,state,AZ,pearson_corr,-0.043351
1,year,search_conducted_false,-0.165978,state,CO,pearson_corr,-0.043351
2,year,search_conducted_false,-0.052226,state,CT,pearson_corr,-0.043351
3,year,search_conducted_false,-0.084114,state,IL,pearson_corr,-0.043351
4,year,search_conducted_false,0.118854,state,MA,pearson_corr,-0.043351


In [15]:
labeled_df.get_subgroup_trends_1lev([corrobj,rankobj,linreg_obj])
labeled_df.result_df.sample(10)

search_conducted_false search_conducted_true
search_conducted_false search_conducted_rate
search_conducted_false contraband_found_false
search_conducted_false contraband_found_true
search_conducted_false contraband_found_rate
search_conducted_false hit_false
search_conducted_false hit_true
search_conducted_false hit_rate
search_conducted_false num_stops
search_conducted_true search_conducted_rate
search_conducted_true contraband_found_false
search_conducted_true contraband_found_true
search_conducted_true contraband_found_rate
search_conducted_true hit_false
search_conducted_true hit_true
search_conducted_true hit_rate
search_conducted_true num_stops
search_conducted_rate contraband_found_false
search_conducted_rate contraband_found_true
search_conducted_rate contraband_found_rate
search_conducted_rate hit_false
search_conducted_rate hit_true
search_conducted_rate hit_rate
search_conducted_rate num_stops
contraband_found_false contraband_found_true
contraband_found_false contraband_fou

contraband_found_false hit_rate
contraband_found_false num_stops
contraband_found_true contraband_found_rate
contraband_found_true hit_false
contraband_found_true hit_true
contraband_found_true hit_rate
contraband_found_true num_stops
contraband_found_rate hit_false
contraband_found_rate hit_true
contraband_found_rate hit_rate
contraband_found_rate num_stops
hit_false hit_true
hit_false hit_rate
hit_false num_stops
hit_true hit_rate
hit_true num_stops
hit_rate num_stops
search_conducted_false search_conducted_true
search_conducted_false search_conducted_rate
search_conducted_false contraband_found_false
search_conducted_false contraband_found_true
search_conducted_false contraband_found_rate
search_conducted_false hit_false
search_conducted_false hit_true
search_conducted_false hit_rate
search_conducted_false num_stops
search_conducted_true search_conducted_rate
search_conducted_true contraband_found_false
search_conducted_true contraband_found_true
search_conducted_true contraband_fou

contraband_found_false hit_false
contraband_found_false hit_true
contraband_found_false hit_rate
contraband_found_false num_stops
contraband_found_true contraband_found_rate
contraband_found_true hit_false
contraband_found_true hit_true
contraband_found_true hit_rate
contraband_found_true num_stops
contraband_found_rate hit_false
contraband_found_rate hit_true
contraband_found_rate hit_rate
contraband_found_rate num_stops
hit_false hit_true
hit_false hit_rate
hit_false num_stops
hit_true hit_rate
hit_true num_stops
hit_rate num_stops
search_conducted_false search_conducted_true
search_conducted_false search_conducted_rate
search_conducted_false contraband_found_false
search_conducted_false contraband_found_true
search_conducted_false contraband_found_rate
search_conducted_false hit_false
search_conducted_false hit_true
search_conducted_false hit_rate
search_conducted_false num_stops
search_conducted_true search_conducted_rate
search_conducted_true contraband_found_false
search_conducte

,feat1,feat2,group_feat,subgroup,subgroup_trend,trend_type,agg_trend
1430,hit_false,num_stops,state,CT,0.881914,pearson_corr,0.846168
520,search_conducted_false,num_stops,driver_race,Black,0.999611,pearson_corr,0.999923
1760,num_stops,driver_race,driver_gender,F,"[Other, Asian, Hispanic, Black, White]",rank_trend,"[Other, Asian, Black, Hispanic, White]"
1006,contraband_found_false,contraband_found_rate,no_reforms,0,-0.0803772,pearson_corr,-0.0740601
2474,search_conducted_rate,contraband_found_rate,medical,0,NaN,lin_reg,NaN
1035,contraband_found_false,hit_false,no_reforms,1,0.818462,pearson_corr,0.845162
1240,contraband_found_true,num_stops,state,SC,0.860348,pearson_corr,0.886805
2351,search_conducted_true,hit_rate,state,WA,4.84795e-06,lin_reg,NaN
2344,search_conducted_true,hit_rate,state,MA,NaN,lin_reg,NaN
2093,search_conducted_false,hit_true,state,MD,NaN,lin_reg,NaN


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

In [16]:
pd.unique(labeled_df.result_df['trend_type'])

array(['pearson_corr', 'rank_trend', 'lin_reg'], dtype=object)